In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import os
import numpy as np
import pandas as pd
import math, random
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
sns.set_style('whitegrid')
from scipy import stats
import missingno as msno

# display all columns
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_row', None)

In [ ]:
# load data
train_data = pd.read_csv('/content/drive/MyDrive/최종프로젝트/physionet2012_train.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/최종프로젝트/physionet2012_valid.csv')
test_data = pd.read_csv('/content/drive/MyDrive/최종프로젝트/physionet2012_test.csv')

train_data.drop(['Unnamed: 0'], axis=1, inplace=True)
valid_data.drop(['Unnamed: 0'], axis=1, inplace=True)
test_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# labeling
# train_data
train_data['label'] = 0
train_data.loc[(2 <= train_data['survival'])&(train_data['survival'] <= train_data['LOS']), 'label'] = 1
train_data.loc[train_data['survival'] > train_data['LOS'], 'label'] = 0
train_data.loc[train_data['survival'] == -1, 'label'] = 0

# valid_data
valid_data['label'] = 0
valid_data.loc[(2 <= valid_data['survival'])&(valid_data['survival'] <= valid_data['LOS']), 'label'] = 1
valid_data.loc[valid_data['survival'] > valid_data['LOS'], 'label'] = 0
valid_data.loc[valid_data['survival'] == -1, 'label'] = 0

# test_data
test_data['label'] = 0
test_data.loc[(2 <= test_data['survival'])&(test_data['survival'] <= test_data['LOS']), 'label'] = 1
test_data.loc[test_data['survival'] > test_data['LOS'], 'label'] = 0
test_data.loc[test_data['survival'] == -1, 'label'] = 0

In [ ]:
train_df = train_data.copy()
valid_df = valid_data.copy()
test_df = test_data.copy()

In [ ]:
input_cols = [x for x in train_df.columns if x not in ['pid', 'Gender', 'ICUType', 'MechVent', 'rec_time', 'Tropl', 'TropT', 'male', 'female', 'survival', 'surv_los', 'label']]

In [ ]:
# ffill
train_df.groupby(['pid']).fillna(method='ffill', inplace=True)
valid_df.groupby(['pid']).fillna(method='ffill', inplace=True)
test_df.groupby(['pid']).fillna(method='ffill', inplace=True)

# median
train_df.fillna(train_df.median(), inplace=True)
valid_df.fillna(valid_df.median(), inplace=True)
test_df.fillna(test_df.median(), inplace=True)

In [ ]:
# Gender -1값 1로 바꾸기
train_df.loc[train_df['Gender']==-1, 'Gender'] = 1 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_df[input_cols])
scaled_train = scaler.transform(train_df[input_cols])
scaled_valid = scaler.transform(valid_df[input_cols])
scaled_test = scaler.transform(test_df[input_cols])
# scaled_train_df = pd.DataFrame(data=scaled_train, columns=ordinary_train.feature_names)

In [ ]:
train_df2 = train_df.copy()
valid_df2 = valid_df.copy()
test_df2 = test_df.copy()

In [ ]:
train_df2[input_cols] = scaled_train
valid_df2[input_cols] = scaled_valid
test_df2[input_cols] = scaled_test

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
train_df3 = []
for pid in tqdm(train_df2.pid.unique()):
    pid_df = train_df2.query('pid==@pid')
    for col in input_cols:
        delta_col = f'{col}_delta'
        dt_values = np.array([0]+list(pid_df[col].values[1:] - pid_df[col].values[:-1]))
        pid_df[delta_col] = dt_values
    train_df3.append(pid_df)

  0%|          | 0/3240 [00:00<?, ?it/s]

In [ ]:
valid_df3 = []
for pid in tqdm(valid_df2.pid.unique()):
    pid_df = valid_df2.query('pid==@pid')
    for col in input_cols:
        delta_col = f'{col}_delta'
        dt_values = np.array([0]+list(pid_df[col].values[1:] - pid_df[col].values[:-1]))
        pid_df[delta_col] = dt_values
    valid_df3.append(pid_df)

  0%|          | 0/360 [00:00<?, ?it/s]

In [ ]:
test_df3 = []
for pid in tqdm(test_df2.pid.unique()):
    pid_df = test_df2.query('pid==@pid')
    for col in input_cols:
        delta_col = f'{col}_delta'
        dt_values = np.array([0]+list(pid_df[col].values[1:] - pid_df[col].values[:-1]))
        pid_df[delta_col] = dt_values
    test_df3.append(pid_df)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
train_df3 = pd.concat(train_df3).reset_index(drop=True)
valid_df3 = pd.concat(valid_df3).reset_index(drop=True)
test_df3 = pd.concat(test_df3).reset_index(drop=True)

In [ ]:
def get_padded_df(df):
    padded_df = []
    p = df['pid'].unique()
    max_len = 70
    for ep in tqdm(p):
        if len(df[df['pid']==ep]) >= max_len:
            ep_df = df[df['pid']==ep][:max_len]
            padded_df.append(ep_df)
        else:
            empty_df = pd.DataFrame(index=range(0,70), columns = df.columns) 
            empty_df[['pid']] = ep
            empty_df[['age']] = df[df['pid']==ep]['age'].values[0]
            empty_df[['Gender']] = df[df['pid']==ep]['Gender'].values[0]
            empty_df[['ICUType']] = df[df['pid']==ep]['ICUType'].values[0]
            empty_df[['label']] = df[df['pid']==ep]['label'].values[0]
            empty_df.iloc[-len(df[df['pid']==ep]):] = df[df['pid']==ep]
            empty_df.fillna(0, inplace=True)
            padded_df.append(empty_df)
    return pd.concat(padded_df).reset_index(drop=True)

In [ ]:
pad_train_df = get_padded_df(train_df3)
pad_valid_df = get_padded_df(valid_df3)
pad_test_df = get_padded_df(test_df3)

  0%|          | 0/3240 [00:00<?, ?it/s]

  0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
# pad_train_df.to_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_train_delta.csv')
# pad_valid_df.to_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_valid_delta.csv')
# pad_test_df.to_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_test_delta.csv')

In [ ]:
'''
# train
pad_train_df = pd.DataFrame(columns = train_df.columns)
p = train_df3['pid'].unique()
max_len = 70
for ep in p:
    if len(train_df3[train_df3['pid']==ep]) >= max_len:
        df = train_df3[train_df3['pid']==ep][:max_len]
        pad_train_df = pd.concat([pad_train_df, df])
    else:
        empty_df = pd.DataFrame(index=range(0,70), columns = train_df.columns) 
        empty_df[['pid']] = ep
        empty_df[['age']] = train_df3[train_df3['pid']==ep]['age'].values[0]
        empty_df[['Gender']] = train_df3[train_df3['pid']==ep]['Gender'].values[0]
        empty_df[['ICUType']] = train_df3[train_df3['pid']==ep]['ICUType'].values[0]
        empty_df[['label']] = train_df3[train_df3['pid']==ep]['label'].values[0]
        empty_df.iloc[-len(train_df3[train_df3['pid']==ep]):] = train_df3[train_df3['pid']==ep]
        empty_df.fillna(0, inplace=True)
        pad_train_df = pd.concat([pad_train_df, empty_df])

# valid
pad_valid_df = pd.DataFrame(columns = valid_df.columns)
p = valid_df3['pid'].unique()
max_len = 70
for ep in p:
    if len(valid_df3[valid_df3['pid']==ep]) >= max_len:
        df = valid_df3[valid_df3['pid']==ep][:max_len]
        pad_valid_df = pd.concat([pad_valid_df, df])
    else:
        empty_df = pd.DataFrame(index=range(0,70), columns = valid_df.columns) 
        empty_df[['pid']] = ep
        empty_df[['age']] = valid_df3[valid_df3['pid']==ep]['age'].values[0]
        empty_df[['Gender']] = valid_df3[valid_df3['pid']==ep]['Gender'].values[0]
        empty_df[['ICUType']] = valid_df3[valid_df3['pid']==ep]['ICUType'].values[0]
        empty_df[['label']] = valid_df3[valid_df3['pid']==ep]['label'].values[0]
        empty_df.iloc[-len(valid_df3[valid_df3['pid']==ep]):] = valid_df3[valid_df3['pid']==ep]
        empty_df.fillna(0, inplace=True)
        pad_valid_df = pd.concat([pad_valid_df, empty_df])

# test
pad_test_df = pd.DataFrame(columns = test_df.columns)
p = test_df3['pid'].unique()
max_len = 70
for ep in p:
    if len(test_df3[test_df3['pid']==ep]) >= max_len:
        df = test_df3[test_df3['pid']==ep][:max_len]
        pad_test_df = pd.concat([pad_test_df, df])
    else:
        empty_df = pd.DataFrame(index=range(0,70), columns = test_df.columns) 
        empty_df[['pid']] = ep
        empty_df[['age']] = test_df3[test_df3['pid']==ep]['age'].values[0]
        empty_df[['Gender']] = test_df3[test_df3['pid']==ep]['Gender'].values[0]
        empty_df[['ICUType']] = test_df3[test_df3['pid']==ep]['ICUType'].values[0]
        empty_df[['label']] = test_df3[test_df3['pid']==ep]['label'].values[0]
        empty_df.iloc[-len(test_df3[test_df3['pid']==ep]):] = test_df3[test_df3['pid']==ep]
        empty_df.fillna(0, inplace=True)
        pad_test_df = pd.concat([pad_test_df, empty_df])
'''

In [ ]:
# load data
pad_train_df = pd.read_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_train_delta.csv')
pad_valid_df = pd.read_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_valid_delta.csv')
pad_test_df = pd.read_csv('/content/drive/MyDrive/최종프로젝트/preprocessed_test_delta.csv')

pad_train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pad_valid_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pad_test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
delta_columns = [x for x in pad_train_df.columns if x not in ['pid', 'rec_time', 'Gender', 'ICUType', 'MechVent', 'Tropl', 'TropT', 'male', 'female', 'survival', 'surv_los', 'LOS', 'label']]

In [ ]:
important_columns = ['age', 'AST', 'Albumin', 'BUN', 'Bilirubin', 'Creatinine', 'GCS', 'Lactate', 'Platelets', 'WBC']

In [ ]:
important_delta_columns = ['age', 'AST', 'Albumin', 'BUN', 'Bilirubin', 'Creatinine', 'GCS', 'Lactate', 'WBC', 'Platelets', 'AST_delta', 'Albumin_delta', 'BUN_delta', 'Bilirubin_delta', 'Creatinine_delta', 'GCS_delta', 'Lactate_delta', 'Platelets_delta', 'WBC_delta']

In [ ]:
vital_columns = ['age', 'GCS', 'HR', 'RR', 'SPO2', 'BT', 'SBP', 'DBP', 'MBP', 'LOS']

In [ ]:
vital_delta_columns = ['age', 'GCS', 'HR', 'RR', 'SPO2', 'BT', 'SBP', 'DBP', 'MBP', 'LOS', 'GCS_delta', 'HR_delta', 'RR_delta', 'SPO2_delta', 'BT_delta', 'SBP_delta', 'DBP_delta', 'MBP_delta', 'LOS_delta']

In [ ]:
vital_fi_columns = ['age', 'AST', 'Albumin', 'BUN', 'Bilirubin', 'Creatinine', 'GCS', 'HR', 'Lactate', 'Platelets', 'RR', 'SPO2', 'BT', 'WBC', 'SBP', 'DBP', 'MBP', 'LOS']

In [ ]:
vital_fi_delta_columns = ['age', 'AST', 'Albumin', 'BUN', 'Bilirubin', 'Creatinine', 'GCS', 'HR', 'Lactate', 'Platelets', 'RR', 'SPO2', 'BT', 'WBC', 'SBP', 'DBP', 'MBP', 'LOS',  'AST_delta', 'Albumin_delta', 'BUN_delta', 'Bilirubin_delta', 'Creatinine_delta', 'GCS_delta', 'HR_delta', 'Lactate_delta', 'Platelets_delta', 'RR_delta', 'SPO2_delta', 'BT_delta', 'WBC_delta', 'SBP_delta', 'DBP_delta', 'MBP_delta', 'LOS_delta']

In [ ]:
# splitting dataset(original)
train_X = pad_train_df[input_cols].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[input_cols].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[input_cols].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original +delta)
train_X = pad_train_df[delta_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[delta_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[delta_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original_importance)
train_X = pad_train_df[important_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[important_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[important_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original + delta _importance)
train_X = pad_train_df[important_delta_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[important_delta_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[important_delta_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original vita)
train_X = pad_train_df[vital_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[vital_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[vital_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original vital + delta)
train_X = pad_train_df[vital_delta_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[vital_delta_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[vital_delta_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original vital importance)
train_X = pad_train_df[vital_fi_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[vital_fi_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[vital_fi_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# splitting dataset(original vital + delta importance)
train_X = pad_train_df[vital_fi_delta_columns].values
train_y = pad_train_df['label'].values

valid_X = pad_valid_df[vital_fi_delta_columns].values
valid_y = pad_valid_df['label'].values

test_X = pad_test_df[vital_fi_delta_columns].values
test_y = pad_test_df['label'].values

# type
train_X = train_X.astype(float)
train_y = train_y.astype(int)

valid_X = valid_X.astype(float)
valid_y = valid_y.astype(int)

test_X = test_X.astype(float)
test_y = test_y.astype(int)

In [ ]:
# imbalanced-learn 패키지
from imblearn.over_sampling import *
from imblearn.over_sampling import SMOTE

# 검증 데이터나 테스트 데이터가 아닌 학습데이터에서만 오버샘플링 사용할 것
# smote = SMOTE(random_state=0)
train_X_over, train_y_over = SMOTE(random_state=0).fit_resample(train_X, train_y)

In [ ]:
# reshape
print(train_X_over.shape, train_y_over.shape)
print(valid_X.shape, valid_y.shape)
print(test_X.shape, test_y.shape)

max_len = 70

train_X_3d = train_X_over.reshape(-1, max_len, train_X_over.shape[-1])
train_y_3d = train_y_over[::70]
valid_X_3d = valid_X.reshape(-1, max_len, valid_X.shape[-1])
valid_y_3d = valid_y[::70]
test_X_3d = test_X.reshape(-1, max_len, test_X.shape[-1])
test_y_3d = test_y[::70]

print(train_X_3d.shape, train_y_3d.shape)
print(valid_X_3d.shape, valid_y_3d.shape)
print(test_X_3d.shape, test_y_3d.shape)

(393820, 35) (393820,)
(25200, 35) (25200,)
(28000, 35) (28000,)
(5626, 70, 35) (5626,)
(360, 70, 35) (360,)
(400, 70, 35) (400,)


In [ ]:
feature_dim = train_X_over.shape[-1]

In [ ]:
# setup
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score

from keras import layers
from keras import Sequential, Model
# from keras.preprocessing.sequence import pad_sequences
from pprint import pprint

from keras.models import Sequential             
from keras.layers import Dense, Dropout, BatchNormalization, SimpleRNN, LSTM, Bidirectional, BatchNormalization, Activation, GRU, Flatten, Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping 
from keras.optimizers import RMSprop, Adam
from keras import models
from keras import layers
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import SimpleRNN, LSTM, GRU

In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_len, feature_dim)))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(8, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve="ROC", name='auroc')])

In [ ]:
# cnn2(maxpooling)
cnn2_model = Sequential()
cnn2_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(max_len, feature_dim)))
cnn2_model.add(MaxPooling1D())
cnn2_model.add(Conv1D(64, kernel_size=3, activation='relu'))
cnn2_model.add(Flatten())
cnn2_model.add(Dense(16, activation='relu'))
cnn2_model.add(BatchNormalization())
cnn2_model.add(Dropout(0.2))
cnn2_model.add(Dense(1, activation='sigmoid'))
cnn2_model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve="ROC", name='auroc')])

In [ ]:
import os

model_dir = '/content/drive/MyDrive/최종프로젝트/model5/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

modelpath='/content/drive/MyDrive/최종프로젝트/model5/{epoch:02d}-{val_auroc:.4f}.hdf5'

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
early_stop = EarlyStopping(monitor='val_auroc', mode='max', patience=10, verbose=1)
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_auroc', verbose=1, mode='max', save_best_only=True)  # callbacks=[checkpointer]

In [ ]:
cnn_model_history = cnn_model.fit(train_X_3d, train_y_3d, epochs=100, batch_size=50, validation_data=(valid_X_3d, valid_y_3d), verbose=1)
cnn_history = cnn_model_history.history

Epoch 1/100
113/113 [==============================] - 5s 29ms/step - loss: 0.6970 - auroc: 0.5007 - val_loss: 0.6971 - val_auroc: 0.4510
Epoch 2/100
113/113 [==============================] - 3s 25ms/step - loss: 0.6962 - auroc: 0.5004 - val_loss: 0.6990 - val_auroc: 0.4288
Epoch 3/100
113/113 [==============================] - 3s 25ms/step - loss: 0.6930 - auroc: 0.5235 - val_loss: 0.7020 - val_auroc: 0.4791
Epoch 4/100
113/113 [==============================] - 3s 25ms/step - loss: 0.6941 - auroc: 0.5084 - val_loss: 0.7008 - val_auroc: 0.5220
Epoch 5/100
113/113 [==============================] - 3s 25ms/step - loss: 0.6940 - auroc: 0.5091 - val_loss: 0.6975 - val_auroc: 0.5286
Epoch 6/100
113/113 [==============================] - 3s 26ms/step - loss: 0.6944 - auroc: 0.5074 - val_loss: 0.6953 - val_auroc: 0.5148
Epoch 7/100
113/113 [==============================] - 3s 24ms/step - loss: 0.6925 - auroc: 0.5208 - val_loss: 0.6959 - val_auroc: 0.5090
Epoch 8/100
113/113 [=============

In [ ]:
cnn2_model_history = cnn2_model.fit(train_X_3d, train_y_3d, epochs=100, batch_size=50, validation_data=(valid_X_3d, valid_y_3d), verbose=1)
cnn2_history = cnn2_model_history.history

Epoch 1/100
113/113 [==============================] - 5s 31ms/step - loss: 0.6753 - auroc: 0.6334 - val_loss: 0.6808 - val_auroc: 0.6305
Epoch 2/100
113/113 [==============================] - 3s 23ms/step - loss: 0.5248 - auroc: 0.8342 - val_loss: 0.6510 - val_auroc: 0.6385
Epoch 3/100
113/113 [==============================] - 2s 22ms/step - loss: 0.4590 - auroc: 0.8894 - val_loss: 0.6235 - val_auroc: 0.6487
Epoch 4/100
113/113 [==============================] - 3s 24ms/step - loss: 0.4142 - auroc: 0.9174 - val_loss: 0.5610 - val_auroc: 0.6573
Epoch 5/100
113/113 [==============================] - 3s 22ms/step - loss: 0.3810 - auroc: 0.9310 - val_loss: 0.5024 - val_auroc: 0.6625
Epoch 6/100
113/113 [==============================] - 2s 22ms/step - loss: 0.3537 - auroc: 0.9390 - val_loss: 0.4745 - val_auroc: 0.6624
Epoch 7/100
113/113 [==============================] - 3s 23ms/step - loss: 0.3321 - auroc: 0.9443 - val_loss: 0.4498 - val_auroc: 0.6638
Epoch 8/100
113/113 [=============

In [ ]:
# predict
cnn_pred = cnn_model.predict(test_X_3d)
cnn2_pred = cnn2_model.predict(test_X_3d)

13/13 [==============================] - 0s 6ms/step


In [ ]:
# score(original_dynamic)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.7207670203359858
CNN2_ROC AUC Score:  0.7607758620689656


In [ ]:
# score(original+delta_dynamic)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.784261715296198
CNN2_ROC AUC Score:  0.8032161803713528


In [ ]:
# score(original feature_importance)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.7723806366047746
CNN2_ROC AUC Score:  0.7287798408488063


In [ ]:
# score(original + delta feature_importance)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.7839301503094607
CNN2_ROC AUC Score:  0.7380083996463307


In [ ]:
# score(original vital)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.6772214854111406
CNN2_ROC AUC Score:  0.7282272325375774


In [ ]:
# score(original + delta_vital)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.7210985853227232
CNN2_ROC AUC Score:  0.736240053050398


In [ ]:
# score(original vital + feature_importance)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.7430923961096375
CNN2_ROC AUC Score:  0.747236958443855


In [ ]:
# score(original vital + delta + feature_importance)
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, accuracy_score, roc_auc_score, average_precision_score
print('CNN_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn_pred))
print('CNN2_ROC AUC Score: ', roc_auc_score(test_y_3d, cnn2_pred))

CNN_ROC AUC Score:  0.7227564102564104
CNN2_ROC AUC Score:  0.7548629531388152
